# Дообучение модели для вопросно-ответной задачи

In [1]:
import operator
import transformers
import torch
import pandas as pd
import pytorch_lightning as pl

from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import (
    AutoTokenizer,
    pipeline,
    AutoModelForQuestionAnswering,
    default_data_collator,
    get_linear_schedule_with_warmup,
)

/home/misha/.pyenv/versions/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Дообучим модель *distilroberta-base* на данных *SQuAD* для *задачи извлечения ответов из текста (Question Answering)*.

*distilroberta-base* - это уменьшенная и оптимизированная версия модели RoBERTa, сохраняющая высокое качество, но работающая быстрее.

*SQuAD* - популярный датасет для оценки производительности моделей в задаче вопросно-ответных систем, содержащий вопросы и ответы на основе википедийных статей.

Что нужно сделать:
1. Выбрать предобученную модель
2. Загрузить соответствующий токенизатор для выбранной модели
3. Разметить и векторизовать данные
4. Загрузить предобученную модель
5. Дообучить на новых данных под свою задачу.

## Подготовка последовательностей

In [2]:
data = load_dataset("squad")

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [4]:
pd.DataFrame(
    data["train"][0, 1, 2, 100, 101, 102],
    columns=["context", "question", "answers"],
)

,context,question,answers
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,One of the main driving forces in the growth o...,In what year did the team lead by Knute Rockne...,"{'text': ['1925'], 'answer_start': [354]}"
4,One of the main driving forces in the growth o...,How many years was Knute Rockne head coach at ...,"{'text': ['13'], 'answer_start': [251]}"
5,One of the main driving forces in the growth o...,How many national titles were won when Knute R...,"{'text': ['three'], 'answer_start': [274]}"


In [5]:
model_name = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Кодирование одной строки

In [6]:
t = "Where can I find a pizzeria?"
print(tokenizer.encode(t))

[0, 13841, 64, 38, 465, 10, 26432, 6971, 116, 2]


Кодирование батча. Так как есть только один образец, то `attention_mask` заполнен `1` (не дополнен нулями (not padded)).

In [7]:
encoded_t = tokenizer(t)
print(encoded_t)

{'input_ids': [0, 13841, 64, 38, 465, 10, 26432, 6971, 116, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


`convert_ids_to_tokens` из массива индексов получает токены. Токенизатор `distilrobera-base` в начале и в конце последовательности ставит специальные токены начала и конца (`<s>` и `</s>`). Также ставит символ `Ġ` перед целыми словом.

In [8]:
print(tokenizer.convert_ids_to_tokens(encoded_t["input_ids"]))

['<s>', 'Where', 'Ġcan', 'ĠI', 'Ġfind', 'Ġa', 'Ġpizz', 'eria', '?', '</s>']


Нам нужно кодировать и вопрос и ответ как пару. Можем строки передавать попарна.

In [9]:
encoded_pair = tokenizer("this is a question", "this is the context")
print(encoded_pair)

{'input_ids': [0, 9226, 16, 10, 864, 2, 2, 9226, 16, 5, 5377, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [10]:
print(tokenizer.convert_ids_to_tokens(encoded_pair["input_ids"]))

['<s>', 'this', 'Ġis', 'Ġa', 'Ġquestion', '</s>', '</s>', 'this', 'Ġis', 'Ġthe', 'Ġcontext', '</s>']


Есть две версии токенизаторов: 1) реализованный на Python и 2) реализованный на Rust (быстрее).

In [11]:
isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

True

*Roberta* имеет ограничение длины последовательности в *512* токенов. Мы можем настраивать этот параметр.

In [12]:
context = "Sarah went to The Mirthless Cafe last night to meet her friend."
question = "Where did Sarah go?"

# The answer span and the answer's starting character position in the context.
answer = "The Mirthless Cafe"
answer_start = 14

In [13]:
x = tokenizer(question, context)
x

{'input_ids': [0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 1672, 16542, 94, 363, 7, 972, 69, 1441, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenizer.batch_decode(x["input_ids"])

['<s>',
 'Where',
 ' did',
 ' Sarah',
 ' go',
 '?',
 '</s>',
 '</s>',
 'Sarah',
 ' went',
 ' to',
 ' The',
 ' M',
 'irth',
 'less',
 ' Cafe',
 ' last',
 ' night',
 ' to',
 ' meet',
 ' her',
 ' friend',
 '.',
 '</s>']

In [15]:
example_max_length = 15
x = tokenizer(
    question, context, max_length=example_max_length, truncation="only_second"
)
x

{'input_ids': [0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Проблема в том, что ответ может обрезаться или вовсе не включаться в последовательность.

In [16]:
tokenizer.batch_decode(x["input_ids"])

['<s>',
 'Where',
 ' did',
 ' Sarah',
 ' go',
 '?',
 '</s>',
 '</s>',
 'Sarah',
 ' went',
 ' to',
 ' The',
 ' M',
 'irth',
 '</s>']

Чтобы гарантировать токенизацию всех токенов контекста с соблюдением максимальной длины, мы можем установить для параметра *return_overflowing_tokens* значение True. Конечный эффект заключается в разделении входных данных на несколько пар "вопрос/контекст", где каждая последующая последовательность контекста является продолжением предыдущей. Поскольку последняя из них может быть короче максимальной длины, мы также устанавливаем длину заполнения справа (padding).

То, что мы получаем в ответ, - это несколько последовательностей *input_id*.

In [17]:
x = tokenizer(
    question,
    context,
    max_length=example_max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    padding="max_length",
)
x

{'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 69, 1441, 4, 2, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]], 'overflow_to_sample_mapping': [0, 0, 0]}

In [18]:
len(x["input_ids"])

3

In [19]:
tokenizer.batch_decode(x["input_ids"])

['<s>Where did Sarah go?</s></s>Sarah went to The Mirth</s>',
 '<s>Where did Sarah go?</s></s>less Cafe last night to meet</s>',
 '<s>Where did Sarah go?</s></s> her friend.</s><pad><pad><pad>']

- В последней последовательности attention_mask присутствуют нули, обозначающие заполнение (padding).
- Массив overflow_to_sample_mapping показывает, из какой пары "вопрос/контекст" произошла каждая последовательность input_ids. В нашем примере мы токенизировали одну пару "вопрос/контекст", что привело к созданию трёх последовательностей input_ids, поэтому overflow_to_sample_mapping состоит из трёх нулей.
- Если бы мы токенизировали две пары "вопрос/контекст", мы бы увидели, что overflow_to_sample_mapping отражает это.

In [20]:
tokenizer(
    ["question 1", "question 2"],
    ["context 1", "context 2"],
    return_overflowing_tokens=True,
)

{'input_ids': [[0, 40018, 112, 2, 2, 46796, 112, 2], [0, 40018, 132, 2, 2, 46796, 132, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 1]}

Однако здесь всё ещё остаётся проблема, заключающаяся в том, что ни одна из последовательностей не содержит полный ответ ("The Mirthless Cafe"). В данном случае правильный полный ответ разделён между последовательностями.

Чтобы устранить это, мы можем токенизировать наши пары "вопрос/контекст" в перекрывающиеся последовательности, установив длину шага (stride).

In [21]:
stride = 5
x = tokenizer(
    question,
    context,
    max_length=example_max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=stride,
    padding="max_length",
)

Установив шаг (stride), равный 5, каждая последующая последовательность контекста начинается на 5 подслов раньше относительно предыдущей последовательности.

Таким образом, две из наших токенизированных последовательностей теперь содержат полный ответ.

In [22]:
tokenizer.batch_decode(x["input_ids"])

['<s>Where did Sarah go?</s></s>Sarah went to The Mirth</s>',
 '<s>Where did Sarah go?</s></s> went to The Mirthless</s>',
 '<s>Where did Sarah go?</s></s> to The Mirthless Cafe</s>',
 '<s>Where did Sarah go?</s></s> The Mirthless Cafe last</s>',
 '<s>Where did Sarah go?</s></s> Mirthless Cafe last night</s>',
 '<s>Where did Sarah go?</s></s>irthless Cafe last night to</s>',
 '<s>Where did Sarah go?</s></s>less Cafe last night to meet</s>',
 '<s>Where did Sarah go?</s></s> Cafe last night to meet her</s>',
 '<s>Where did Sarah go?</s></s> last night to meet her friend</s>',
 '<s>Where did Sarah go?</s></s> night to meet her friend.</s>']

In [23]:
print(x.keys(), "\n")
x

KeysView({'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 439, 7, 20, 256, 24208, 1672, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 7, 20, 256, 24208, 1672, 16542, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 20, 256, 24208, 1672, 16542, 94, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 256, 24208, 1672, 16542, 94, 363, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 24208, 1672, 16542, 94, 363, 7, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 16542, 94, 363, 7, 972, 69, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 94, 363, 7, 972, 69, 1441, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 363, 7, 972, 69, 1441, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

{'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 439, 7, 20, 256, 24208, 1672, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 7, 20, 256, 24208, 1672, 16542, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 20, 256, 24208, 1672, 16542, 94, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 256, 24208, 1672, 16542, 94, 363, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 24208, 1672, 16542, 94, 363, 7, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 16542, 94, 363, 7, 972, 69, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 94, 363, 7, 972, 69, 1441, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 363, 7, 972, 69, 1441, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 

Для тонкой настройки модели для ответов на вопросы наша предварительно обученная модель *distilroberta-base* ожидает, что этот объект будет содержать ещё и:

- *start_positions*: позиции токенов, где начинаются ответы.
- *end_positions*: позиции токенов, где заканчиваются ответы.

In [24]:
print(answer_start)
print(context[answer_start : answer_start + len(answer)])

14
The Mirthless Cafe


Нам нужно использовать это, чтобы определить позиции <u>токенов</u>, где каждый ответ начинается и заканчивается в каждой последовательности input_ids. В некоторых случаях полный ответ может отсутствовать в конкретной последовательности. Нам также нужно обрабатывать такие ситуации.

Для этого мы получим дополнительную информацию, установив для параметра return_offsets_mapping значение True в токенизаторе.

In [25]:
x = tokenizer(
    question,
    context,
    max_length=example_max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=stride,
    return_offsets_mapping=True,
    padding="max_length",
)
x

{'input_ids': [[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 439, 7, 20, 256, 24208, 1672, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 7, 20, 256, 24208, 1672, 16542, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 20, 256, 24208, 1672, 16542, 94, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 256, 24208, 1672, 16542, 94, 363, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 24208, 1672, 16542, 94, 363, 7, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 1672, 16542, 94, 363, 7, 972, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 16542, 94, 363, 7, 972, 69, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 94, 363, 7, 972, 69, 1441, 2], [0, 13841, 222, 4143, 213, 116, 2, 2, 363, 7, 972, 69, 1441, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 

In [26]:
print(len(x["input_ids"]))
print(len(x["offset_mapping"]))

10
10


Каждый элемент в offset_mapping указывает начальную и конечную позицию символа для каждого токена в исходной строке. Сопоставление смещения (0, 0) представляет специальный токен (например, `<s>`).

Например, вот первая последовательность input_ids вместе с её соответствующим offset_mapping.

In [27]:
print(x["input_ids"][0])
print(x["offset_mapping"][0])

[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2]
[(0, 0), (0, 5), (6, 9), (10, 15), (16, 18), (18, 19), (0, 0), (0, 0), (0, 5), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23), (0, 0)]


In [28]:
print("First non-special input_id converted to token:")
print(tokenizer.convert_ids_to_tokens(x["input_ids"][0][1]), "\n")

offset = x["offset_mapping"][0][1]
print(
    f"Span extracted from context using corresponding offset_mapping {offset}:"
)
print(question[offset[0] : offset[1]])

First non-special input_id converted to token:
Where 

Span extracted from context using corresponding offset_mapping (0, 5):
Where


Поскольку нам известна позиция символа, с которой начинается ответ, мы можем использовать её вместе с *offset_mapping*, чтобы определить начальную и конечную позиции токенов для отрезка ответа.

Единственная оставшаяся проблема — это определение того, относится ли смещение к вопросу или к контексту. Если посмотреть на первые два *offset_mapping*, можно заметить, что:

1.  В первой последовательности оба *offset_mapping* (как для вопроса, так и для контекста) начинаются с нуля.
2.  Во второй последовательности значения *offset_mapping* для контекста продолжаются с предыдущей последовательности (с учётом заданного шага).

In [29]:
print(x["offset_mapping"][0])
print(x["offset_mapping"][1])

[(0, 0), (0, 5), (6, 9), (10, 15), (16, 18), (18, 19), (0, 0), (0, 0), (0, 5), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23), (0, 0)]
[(0, 0), (0, 5), (6, 9), (10, 15), (16, 18), (18, 19), (0, 0), (0, 0), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23), (23, 27), (0, 0)]


Это означает, что нам необходимо определить:

1. Какие из *offset_mapping* относятся к контексту.
2. Содержит ли конкретная последовательность ответ вообще.

Первая задача может быть выполнена с помощью метода *sequence_ids* для объекта кодирования. Каждая последовательность *input_ids* имеет соответствующий список *sequence_ids*, который указывает, является ли токен частью вопроса, частью контекста или специальным токеном.

In [30]:
print(x["input_ids"][0])
print(x.sequence_ids(0))

[0, 13841, 222, 4143, 213, 116, 2, 2, 33671, 439, 7, 20, 256, 24208, 2]
[None, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, None]


Таким образом, чтобы определить, является ли токен частью контекста, мы можем использовать sequence_ids и проверить, соответствует ли позиция токена значению 1.

Для решения второй проблемы мы можем проверить, находятся ли начальная и конечная позиции символов ответа в пределах наименьшего и наибольшего значений сопоставления смещений (offset mapping) соответственно.

In [31]:
# We can calculate the answer end character position using the answer length.
answer_end = answer_start + len(answer)

print("Answer start character position:", answer_start)
print("Answer end character position:", answer_end)
print("Answer pulled from context:", context[answer_start:answer_end])

Answer start character position: 14
Answer end character position: 32
Answer pulled from context: The Mirthless Cafe


In [32]:
tokenizer.batch_decode(x["input_ids"])

['<s>Where did Sarah go?</s></s>Sarah went to The Mirth</s>',
 '<s>Where did Sarah go?</s></s> went to The Mirthless</s>',
 '<s>Where did Sarah go?</s></s> to The Mirthless Cafe</s>',
 '<s>Where did Sarah go?</s></s> The Mirthless Cafe last</s>',
 '<s>Where did Sarah go?</s></s> Mirthless Cafe last night</s>',
 '<s>Where did Sarah go?</s></s>irthless Cafe last night to</s>',
 '<s>Where did Sarah go?</s></s>less Cafe last night to meet</s>',
 '<s>Where did Sarah go?</s></s> Cafe last night to meet her</s>',
 '<s>Where did Sarah go?</s></s> last night to meet her friend</s>',
 '<s>Where did Sarah go?</s></s> night to meet her friend.</s>']

In [33]:
input_ids = x["input_ids"][0]
offset_mapping = x["offset_mapping"][0]
seq_ids = x.sequence_ids(0)

In [34]:
# These are the sequence ids
print("Sequence IDs: ", seq_ids)

Sequence IDs:  [None, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, None]


In [35]:
context_pos_start = seq_ids.index(1)

In [36]:
# Utility function to find the *last* occurrence of a sequence.
def rindex(lst, value):
    return len(lst) - operator.indexOf(reversed(lst), value) - 1


# Get the end index position (i.e. the last occurrence of 1).
context_pos_end = rindex(seq_ids, 1)

In [37]:
print("Context tokens begin at position", context_pos_start)
print("Context tokens end at position", context_pos_end)

Context tokens begin at position 8
Context tokens end at position 13


Теперь, когда мы знаем, какие токены являются частью контекста, мы можем посмотреть на их соответствующие сопоставления смещений (offset mappings), чтобы проверить, находятся ли начальная и конечная позиции символов в пределах этих смещений.

In [38]:
# These are the corresponding offsets.
context_offsets = offset_mapping[context_pos_start : context_pos_end + 1]
print(context_offsets)

[(0, 5), (6, 10), (11, 13), (14, 17), (18, 19), (19, 23)]


In [39]:
print(
    "Is the lowest offset value lower than or equal to the starting character position?"
)
print("Answer starting character position:", answer_start)
print("First offset:", context_offsets[0])

# Note how we're checking the first tuple value.
print(context_offsets[0][0] <= answer_start)

Is the lowest offset value lower than or equal to the starting character position?
Answer starting character position: 14
First offset: (0, 5)
True


In [40]:
print(
    "Is the highest offset value higher than or equal to the ending character position?"
)
print("Answer ending character position:", answer_end)
print("Last offset:", context_offsets[-1])

# Note how how we're checking the second tuple value.
print(context_offsets[-1][1] >= answer_end)

Is the highest offset value higher than or equal to the ending character position?
Answer ending character position: 32
Last offset: (19, 23)
False


Итак, первая последовательность содержит часть ответа, но полный ответ обрезается. Это подтверждается визуальной проверкой:

In [41]:
print(tokenizer.batch_decode(input_ids))

['<s>', 'Where', ' did', ' Sarah', ' go', '?', '</s>', '</s>', 'Sarah', ' went', ' to', ' The', ' M', 'irth', '</s>']


Сделаем тоже самое для третьей последовательности.

In [42]:
input_ids = x["input_ids"][2]
offset_mapping = x["offset_mapping"][2]
seq_ids = x.sequence_ids(2)

context_pos_start = seq_ids.index(1)
context_pos_end = rindex(seq_ids, 1)

context_offsets = offset_mapping[context_pos_start : context_pos_end + 1]

print(
    "Is the lowest offset value lower than or equal to the starting character position?"
)
print("Answer starting character position:", answer_start)
print("First offset:", context_offsets[0])

# Note how we're checking the first tuple value.
print(context_offsets[0][0] <= answer_start)

print(
    "Is the highest offset value higher than or equal to the ending character position?"
)
print("Answer ending character position:", answer_end)
print("Last offset:", context_offsets[-1])

# Note how how we're checking the second tuple value.
print(context_offsets[-1][1] >= answer_end)


Is the lowest offset value lower than or equal to the starting character position?
Answer starting character position: 14
First offset: (11, 13)
True
Is the highest offset value higher than or equal to the ending character position?
Answer ending character position: 32
Last offset: (28, 32)
True


Теперь, когда мы подтвердили, что третья последовательность содержит полный ответ, нам нужно определить, где ответ начинается и заканчивается в *input_ids*. Мы можем сделать это, просканировав offset_mapping слева направо, чтобы найти начало, и справа налево, чтобы найти конец.

In [43]:
s = e = 0

# Начинаем сканировать offset_mapping слева,
# чтобы найти позицию токена, где начинается ответ.
# Нет гарантии, что токенизатор выдаст токен, у которого
# начальный символ совпадает с первым символом ответа. Когда
# это происходит, мы берём позицию предыдущего токена в качестве начала.
i = context_pos_start
while offset_mapping[i][0] < answer_start:
    i += 1
if offset_mapping[i][0] == answer_start:
    s = i
else:
    s = i - 1

# Поиск конечного токена
j = context_pos_end
while offset_mapping[j][1] > answer_end:
    j -= 1
if offset_mapping[j][1] == answer_end:
    e = j
else:
    e = j + 1

In [44]:
print("Answer start token position in context:", s)
print("Answer end token position in context:", e)

Answer start token position in context: 9
Answer end token position in context: 13


In [45]:
print("Answer lifted from context:")
tokenizer.batch_decode(input_ids[s : e + 1])

Answer lifted from context:


[' The', ' M', 'irth', 'less', ' Cafe']

Запишем всю логику в функцию.

In [46]:
max_length = 400
stride = 100
batch_size = 32


def prepare_dataset(examples):
    # Some tokenizers don't strip spaces. If there happens to be question text
    # with excessive spaces, the context may not get encoded at all.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]

    # Tokenize.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # We'll collect a list of starting positions and ending positions.
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # Work through every sequence.
    for seq_idx in range(len(tokenized_examples["input_ids"])):
        seq_ids = tokenized_examples.sequence_ids(seq_idx)
        offset_mappings = tokenized_examples["offset_mapping"][seq_idx]

        cur_example_idx = tokenized_examples["overflow_to_sample_mapping"][
            seq_idx
        ]
        answer = examples["answers"][cur_example_idx]
        answer_text = answer["text"][0]
        answer_start = answer["answer_start"][0]
        answer_end = answer_start + len(answer_text)

        context_pos_start = seq_ids.index(1)
        context_pos_end = rindex(seq_ids, 1)

        s = e = 0
        if (
            offset_mappings[context_pos_start][0] <= answer_start
            and offset_mappings[context_pos_end][1] >= answer_end
        ):
            i = context_pos_start
            while offset_mappings[i][0] < answer_start:
                i += 1
            if offset_mappings[i][0] == answer_start:
                s = i
            else:
                s = i - 1

            j = context_pos_end
            while offset_mappings[j][1] > answer_end:
                j -= 1
            if offset_mappings[j][1] == answer_end:
                e = j
            else:
                e = j + 1

        tokenized_examples["start_positions"].append(s)
        tokenized_examples["end_positions"].append(e)

    return tokenized_examples

In [47]:
max_length = 400
stride = 100
batch_size = 32

In [48]:
tokenized_datasets = data.map(
    prepare_dataset,
    batched=True,
    remove_columns=data["train"].column_names,
    num_proc=2,
)

In [49]:
data = tokenized_datasets.remove_columns(
    ["offset_mapping", "overflow_to_sample_mapping"]
)

In [50]:
train_dataset = data["train"]
val_dataset = data["validation"]

In [51]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=default_data_collator,
    num_workers=4,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    num_workers=4,
)

## Дообучение модели

In [52]:
model_name = "distilroberta-base"
batch_size = 8
lr = 3e-5
num_epochs = 1


class QAModel(pl.LightningModule):
    def __init__(self, model_name, lr):
        super().__init__()
        self.save_hyperparameters()
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        self.model.train()
        self.lr = lr

    def forward(self, **batch):
        return self.model(**batch)

    def on_train_start(self):
        self.model.train()

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.lr)
        num_training_steps = len(train_dataloader) * num_epochs
        lr_scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=num_training_steps,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": lr_scheduler,
                "interval": "step",
                "frequency": 1,
                "reduce_on_plateau": False,
                "monitor": None,
                "strict": False,
            },
        }

In [53]:
model = QAModel(model_name, lr)

trainer = pl.Trainer(
    max_epochs=num_epochs,
    precision="16-mixed",
    gradient_clip_val=1.0,
    accelerator="auto",
    devices="auto",
)

trainer.fit(model, train_dataloader, val_dataloader)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/misha/.pyenv/versions/torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=Tru

Epoch 0:   0%|          | 2/2761 [00:00<09:36,  4.79it/s, v_num=0, train_loss=6.100]

/home/misha/.pyenv/versions/torch/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 0: 100%|██████████| 2761/2761 [04:24<00:00, 10.44it/s, v_num=0, train_loss=0.768, val_loss=1.080]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 2761/2761 [04:26<00:00, 10.35it/s, v_num=0, train_loss=0.768, val_loss=1.080]


## Инференс

In [54]:
# ========== Функция для ответа ==========
@torch.inference_mode
def get_answer(tokenizer, model, question, context):
    inputs = tokenizer(question, context, return_tensors="pt")
    outputs = model(**inputs)
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits)
    answer_ids = inputs["input_ids"][0, start_idx : end_idx + 1]
    return tokenizer.decode(answer_ids, skip_special_tokens=True)


In [55]:
c = "Sarah went to The Mirthless Cafe last night to meet her friend."
q = "Where did Sarah go?"
get_answer(tokenizer, model, q, c)

' The Mirthless Cafe'

In [56]:
q = "Who did Sarah meet?"
get_answer(tokenizer, model, q, c)

' her friend'

In [57]:
q = "When did Sarah meet her friend?"
get_answer(tokenizer, model, q, c)

' last night'

In [58]:
q = "Who went to the restaurant?"
get_answer(tokenizer, model, q, c)

'Sarah'

Но у извлечения ответов из контекста есть свои ограничения

In [59]:
# Задавать логическую загадку сложно, несмотря на то, что
# ответ доступен. По правде говоря, здесь есть двусмысленность.
q = "Who did Sarah's friend meet?"
get_answer(tokenizer, model, q, c)

''

In [60]:
# Модель не может определить, когда на вопрос невозможно
# ответить. В некоторых наборах данных для ответов на вопросы
# этому явно обучают.
q = "How did Sarah get to the restaurant?"
get_answer(tokenizer, model, q, c)

' to meet her friend'

In [61]:
# Модель также не является генеративной.
q = "What is a possible reason for why Sarah met her friend?"
get_answer(tokenizer, model, q, c)

'.'